In [54]:
import pandas as pd
import datetime
import urllib
import numpy as np
from io import BytesIO
from io import StringIO

In [12]:
def get_data(symbol,start,end):
    base_url = 'https://query1.finance.yahoo.com/v7/finance/download/'
    #EJ Symbol: ^GSPC
    base_url = base_url+symbol+'?%s'
    payload = urllib.parse.urlencode({'period1': start, 'period2': end, 'interval': '1d', 'events':'history'})

    req = urllib.request.urlopen(base_url % payload)
    req_read = req.read()
    file_download = BytesIO(req_read)

    df = pd.read_csv(file_download)

    df['Close'].replace('', np.nan, inplace=True)
    df.dropna(subset=['Close'], inplace=True)
    
    df['Symbol'] = symbol
    df['actualizacion'] = datetime.datetime.now()

    return df

In [44]:
def upload_azure():
    #Verificar si la ruta existe.
    #symbols = ['^GSPC','GC=F','BG','SI=F','^DJI','GLEN.L','ADM']

    data = pd.DataFrame()


    symbol = 'USDRUB=X'
    max_date = datetime.datetime(2010, 1, 4, 0, 0) #'2010-01-04'

    start_date = datetime.datetime.combine(max_date, datetime.datetime.min.time()) + datetime.timedelta(days=1)
    end_date = datetime.datetime.now() - datetime.timedelta(days=1)

    date_now_timestamp = str(datetime.datetime.timestamp(start_date)).split('.')
    start = date_now_timestamp[0]

    date_now_timestamp = str(datetime.datetime.timestamp(end_date)).split('.')
    end = date_now_timestamp[0]

    symbol_data = get_data(symbol,start,end)

    if(symbol_data.empty):
        return '{"Result":"False"}'

    file_download = symbol_data.to_csv(header=True,index=False,encoding='utf-8-sig')        
    #name = 'YAHOO_FINANCE.csv'
    
    return file_download

In [45]:
yah = upload_azure()

In [55]:
yah_csv = StringIO(yah)
yah_pd = pd.read_csv(yah_csv, sep=',')

In [56]:
yah_pd.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,actualizacion
0,2010-01-05,29.913000,29.995199,29.905001,29.995199,29.995199,0.0,USDRUB=X,2020-07-16 12:48:19.189815
1,2010-01-06,29.995199,30.051001,29.271000,29.995199,29.995199,0.0,USDRUB=X,2020-07-16 12:48:19.189815
2,2010-01-07,28.812000,29.995199,28.812000,29.995199,29.995199,0.0,USDRUB=X,2020-07-16 12:48:19.189815
3,2010-01-08,29.995199,29.995199,29.830000,29.865999,29.865999,0.0,USDRUB=X,2020-07-16 12:48:19.189815
4,2010-01-11,29.864000,29.875000,29.273800,29.313700,29.313700,0.0,USDRUB=X,2020-07-16 12:48:19.189815


In [57]:
yah_pd.to_csv('YAHOO_FINANCE.csv',index=False)